In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from datetime import datetime
import json

In [2]:
# Obtendo a data e hora atual
now = datetime.now()
formatted_date = now.strftime("%Y_%m_%d_%H_%M")

def extract_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        data = json.load(file)

    point_coords = []
    for feature in data['features']:
        if feature['geometry']['type'] == 'Point':
            point_coords.append(feature['geometry']['coordinates'])

    return point_coords

### Modelo

In [3]:
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = SamGeo(
    model_type="vit_b",
    automatic=False,
    sam_kwargs=None,
)

### Imagem a ser segmentada

In [4]:
# imagem

image = "dados_google_earth/2022_04_recorte.tif"
#image = 'dados_drone/2023_12.tif'


# pasta saída, arquivo saída

pasta_saida = f'saida/mascaras_por_ponto/{os.path.splitext(os.path.basename(image))[0]}'

os.makedirs(pasta_saida, exist_ok=True)

mascaras_geradas = f'{pasta_saida}/mascaras_{formatted_date}.tif'

# reprojetar imagem para 3857

image_3857 = f'{pasta_saida}/{os.path.splitext(os.path.basename(image))[0]}_3857.tif'

!gdalwarp -t_srs EPSG:3857 -r near -of GTiff {image} {image_3857}

Using band 4 of source image as alpha.
Creating output file that is 1461P x 3429L.
Processing dados_google_earth/2022_04_recorte.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


In [5]:
sam.set_image(image_3857)

In [6]:
m = leafmap.Map(center=[-26.9565, -48.6294], zoom=19, height="400px")
# m.add_basemap("SATELLITE")

m.add_raster(image_3857, layer_name="2009_08")

In [7]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…